# Basic Simulation from PDB

The starting configuration for this vignette comes from the Toolkit Showcase, which assembles and solvates this topology starting from a prepared protein PDB and ligand SDF. 

https://github.com/openforcefield/openff-toolkit/tree/main/examples/toolkit_showcase

This notebook describes the basic workflow for any OpenFF simulation.

## OpenFF Code

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

ligand = Molecule.from_file("inputs/PT2385.sdf")
top = Topology.from_pdb("inputs/solvated_complex.pdb", unique_molecules=[ligand])
ff = ForceField("openff-2.1.1.offxml", "ff14sb_off_impropers_0.0.3.offxml")

ic = ff.create_interchange(top)
ic.visualize()

## OpenMM Code

In [ ]:
import openmm
from openff.units import Quantity, unit

simulation = ic.to_openmm_simulation(
    openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtoseconds,
    )
)

# Energy minimize
simulation.minimizeEnergy()

# Add a reporter to record the structure every 1000 steps
dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 1000)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())
simulation.runForClockTime(1 * openmm.unit.minute)

## NGLView

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(top.to_openmm())
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="water")
view.add_representation(
    "hyperball", radiusSize=1, radiusScale=0.5, selection="not protein and not water"
)
view